# le-livre

---

In [2]:
import pandas as pd
import os
import mysql.connector

In [3]:
try:
    connection = mysql.connector.connect(host='***',
                                         database='***',
                                         user='***',
                                         password='***',
                                         port='***')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        ### Dataframe 1
        var_sql = "SELECT date_pan as purchase_date, n_article as sku, panier_v.Prix as item_price, iso as ship_country FROM db_lelivre_pro.panier_v JOIN com_v ON com_v.UID=panier_v.n_com JOIN zones ON zones.Idpays=com_v.Pays WHERE date_pan > \"2018-12-31 23:59:59\" AND date_pan < \"2020-01-01 00:00:00\" AND etat = 4"
        df_raw = pd.read_sql(var_sql, connection)
        
        print("Dataframes constitués")
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.18
You're connected to database:  ('db_lelivre_pro',)
Dataframes constitués
MySQL connection is closed


In [4]:
df_raw.shape

(3713, 4)

In [5]:
df_raw.head()

,purchase_date,sku,item_price,ship_country
0,2019-01-01 06:40:08,R130001251,10.95,FR
1,2019-01-01 06:40:08,R130004489,14.90,FR
2,2019-01-01 06:40:08,R130011303,14.90,FR
3,2019-01-01 14:48:46,R200033488,59.25,BE
4,2019-01-01 15:00:00,R200001802,49.50,FR


In [6]:
df_raw.dtypes

purchase_date    datetime64[ns]
sku                      object
item_price              float64
ship_country             object
dtype: object

In [7]:
df_raw['purchase_date'] = df_raw['purchase_date'].astype(str).str.slice(0,10)
df_raw['purchase_date'] = pd.to_datetime(df_raw['purchase_date'],format="%Y-%m-%d")
df_raw

,purchase_date,sku,item_price,ship_country
0,2019-01-01,R130001251,10.95,FR
1,2019-01-01,R130004489,14.90,FR
2,2019-01-01,R130011303,14.90,FR
3,2019-01-01,R200033488,59.25,BE
4,2019-01-01,R200001802,49.50,FR
...,...,...,...,...
3708,2019-12-31,RO20117242,10.95,US
3709,2019-12-31,RO20118319,14.90,US
3710,2019-12-31,RO20117808,10.95,US
3711,2019-12-31,RO20118339,14.90,US


In [8]:
df_raw.dtypes

purchase_date    datetime64[ns]
sku                      object
item_price              float64
ship_country             object
dtype: object

In [9]:
os.chdir("C:\Adrian - GDrive\Professionnel\Formation\Informatique - Digital\Data University\Projet certification\src")
df_country = pd.read_csv("sql-pays.csv")
df_country

,Id,Code,Alpha2,Alpha3,nom_fr_fr,nom_en_gb
0,1,4,AF,AFG,Afghanistan,Afghanistan
1,2,8,AL,ALB,Albanie,Albania
2,3,10,AQ,ATA,Antarctique,Antarctica
3,4,12,DZ,DZA,Algérie,Algeria
4,5,16,AS,ASM,Samoa Américaines,American Samoa
...,...,...,...,...,...,...
236,237,876,WF,WLF,Wallis et Futuna,Wallis and Futuna
237,238,882,WS,WSM,Samoa,Samoa
238,239,887,YE,YEM,Yémen,Yemen
239,240,891,CS,SCG,Serbie-et-Monténégro,Serbia and Montenegro


In [12]:
df_raw = df_raw.merge(df_country, left_on='ship_country', right_on='Alpha2')
df_raw.columns

Index(['purchase_date', 'sku', 'item_price', 'ship_country', 'Id', 'Code',
       'Alpha2', 'Alpha3', 'nom_fr_fr', 'nom_en_gb'],
      dtype='object')

In [13]:
df_raw = df_raw.drop(columns=['ship_country', 'Id', 'Code', 'Alpha2', 'Alpha3', 'nom_fr_fr'])
df_raw

,purchase_date,sku,item_price,nom_en_gb
0,2019-01-01,R130001251,10.95,France
1,2019-01-01,R130004489,14.90,France
2,2019-01-01,R130011303,14.90,France
3,2019-01-01,R200001802,49.50,France
4,2019-01-02,R300231701,19.80,France
...,...,...,...,...
3707,2019-11-08,RO90017665,10.95,Réunion
3708,2019-12-15,R200048603,19.80,Réunion
3709,2019-11-17,RO30018207,14.90,French Guiana
3710,2019-11-17,RO30082154,29.80,French Guiana


In [14]:
os.chdir("C:\Adrian - GDrive\Professionnel\Formation\Informatique - Digital\Data University\Projet certification\csv")

df_raw.to_csv("1-lelivre.csv", index=False)